# Generalized Sequential Pattern
Der Generalized Sequential Pattern (GSP) Algorithmus ist ein beliebter Algorithmus im Bereich des Data Mining, der verwendet wird, um häufige Muster oder Sequenzen in einer Datenmenge zu finden. Der Algorithmus verwendet eine Monotonie-Eigenschaft und ist verwandt mit dem bereits bekannten Apriori-Algorithmus.

In [ ]:
from tui_dsmt.fpm import SequentialItemset, dna

## Inhaltsverzeichnis
- [Wiederholung: Monotonie-Eigenschaft](#Wiederholung-Monotonie-Eigenschaft)
- [Der Algorithmus](#Der-Algorithmus)
- [Zusammenfassung](#Zusammenfassung)

## Wiederholung: Monotonie-Eigenschaft
Die Monotonie-Eigenschaft besagte, dass jede Teilmenge eines häufig auftretenden Itemsets selbst häufig sein muss. Umgekehrt bedeutet dies, dass ein nicht-häufiges Itemset nicht mit einem beliebigen anderen Itemset zu einem häufigen, größeren Itemset vereinigt werden kann. Während des Apriori-Algorithmus konnte durch Ausschluss kleiner Itemsets somit eine Reduzierung des Suchraums auch im Hinblick auf größere Itemsets erreicht werden.

Das Prinzip lässt sich ebenfalls auf Sequenzen anwenden. Jede Subsequenz einer häufig auftretenden Sequenz muss ebenfalls häufig sein.

## Der Algorithmus
Betrachten Sie zuerst den Datensatz, der beispielhaft verwendet werden soll. Enthalten sind drei verschiedene (fiktive) DNA-Sequenzen, die im Folgenden auf gemeinsame Muster untersucht werden sollen.

In [ ]:
dna

Der Algorithmus benötigt außerdem einen vorgegebenen, minimalen Support.

In [ ]:
min_supp = 3

### Bestimmung der einelementigen Sequenzen
Analog zum Apriori-Algorithmus startet der GSP-Algorithmus mit der Suche nach Mustern der Länge $1$ und filtert diese mit Hilfe des minimalen Supports.

In [ ]:
def itemsets1(transactions, min_supp):
    # leere Abbildung anlegen
    C1 = {}

    # alle Transaktionen durchgehen
    for _, items in transactions:
        # alle Elemente der Transaktion betrachten
        for element in items:
            # Element zur Abbildung hinzufügen
            # bzw. Anzahl um 1 erhöhen
            if element not in C1:
                C1[element] = 1
            else:
                C1[element] += 1

    # Elemente nach minimalem Support filtern
    # und in Itemset umwandeln
    return set(SequentialItemset(item) for item, count in C1.items()
               if count >= min_supp)


L1 = itemsets1(dna, min_supp)
L1

Kandidatenmengen werden dabei wieder mit $C$ bezeichnet, während die zum selben Schritt gehörenden Mengen häufiger Sequenzen mit $F$ (statt $L$) bezeichnet werden.

### Generierung größerer Kandidatenmengen
Während beim Apriori-Algorithmus immer zwei $k-1$-Itemsets, die sich unter Zuhilfenahme einer lexikographischen Ordnung in den ersten $k-2$ Elementen unterscheiden, zu einem $k$-Itemset zusammengeführt wurden, muss dieses Vorgehen beim GSP-Algorithmus angepasst werden. Ziel ist es jedoch erneut, aus mehreren $k-1$-Sequenzen neue $k$-Sequenzen zu erzeugen - und davon erneut möglichst wenige, die im weiteren Verlauf verworfen werden.

**Beispiel**:

Sei $L_2 = \{ (A, G), (A, T), (G, T) \}$.

Dann ist im Apriori-Algorithmus klar, dass als Kombination $(A, G, T)$ entsteht und $(A, G, T) \in C_3$ gilt.

Wird nicht nur nach häufigem gemeinsamen Auftreten gefragt, sondern Sequenzen unter Beibehaltung ihrer Reihenfolge gesucht, ergeben sich mehrere potentielle Kombinationen.
- Da zwei der Sequenzen mit $A$ beginnen, wird eine entstehende Sequenz ebenfalls mit $A$ beginnen. An zweiter Stelle folgt dann **entweder** $G$ **oder** $T$, sodass für die dritte Stelle jeweils der nicht gewählte Buchstabe verbleibt. Es entstehen die Sequenzen $<\!A, G, T\!>$ und $<\!A, T, G\!>$.
- Aus $<\!A, T\!>$ und $<\!G, T\!>$ entstehen mit ähnlicher Argumentation die Sequenzen $<\!A, G, T\!>$ und $<\!G, A, T\!>$.
- Aus $<\!A, G\!>$ und $<\!G, T\!>$ entsteht die Sequenz $<\!A, G, T\!>$.

Um keine unnötigen oder doppelten Sequenzen zu erzeugen, wird die **Join-Phase** wie folgt ausgeführt: Die Kandidatenmenge $C_k$ wird durch einen Verbund aus $L_{k-1}$ und $L_{k-1}$ erzeugt. Seien dafür $s_1, s_2 \in L_{k-1}$. Ein neues Element $s_3$ wird erzeugt,
- falls $s_1[2:k-1] = s_2[1:k-2]$ bzw. die erste Sequenz ohne das erste Item mit der zweiten Sequenz ohne das letzte Item übereinstimmt,
- indem $s_3 = s_1 + s_2[k-1:k-1]$ bzw. das letzte Element von $s_2$ an $s_1$ angehangen wird.

Analog zum Apriori-Algorithmus existiert eine angehängte **Pruning-Phase**, die Kandidaten entfernt, die bereits ausgeschlossene Teilsequenzen enthalten. Dafür wird eine Sequenz der Länge $k$ wieder ausgeschlossen, falls sie eine $k-1$-Teilsequenz beinhaltet, die nicht in $L_{k-1}$ enthalten ist und damit nicht den minimalen Support erfüllt.

Die nachfolgende Zelle formuliert die beiden zuvor genannten Schritte in Python. Beachten Sie bitte, dass die Zählung in Python bei $0$ beginnt und der Endwert einer Slicing Operation nicht im Ergebnis enthalten ist, sodass die Indizes für Prä- und Suffixe jeweils angepasst werden müssen. (In Python können außerdem negative oder leere Start- und Endwerte verwendet werden. Wir verzichten hier darauf, um näher an der Definition zu bleiben.)

In [ ]:
def generiere_kandidaten(L, k):
    # Join
    all_C_k = set()

    for s1 in L:
        for s2 in L:
            if s1[1:k-1] == s2[0:k-2]:
                new_s = s1 + s2[k-2:k-1]
                all_C_k.add(new_s)

    # Pruning
    C_k = set()

    for c in all_C_k:
        for subset in c.subsets(k-1):
            if subset not in L:
                break
        else:
            C_k.add(c)

    return C_k


C2 = generiere_kandidaten(L1, 2)
C2

### Filtern nach minimalem Support
Um $C_k$ zu $L_k$ zu reduzieren, wird erneut ein Scan der Datenbank durchgeführt und der Support aller in $C_k$ enthaltenen, $k$-elementigen Teilmengen mit dem vorgegebenen, minimalen Support verglichen und entsprechend gefiltert.

In [ ]:
def filtere_kandidaten(C_k, transactions, min_supp):
    return set(c for c in C_k if c.count_in(transactions) >= min_supp)


L2 = filtere_kandidaten(C2, dna, min_supp)
L2

### Finaler Algorithmus
Wie bereits zuvor werden die Bausteine zum finalen Algorithmus zusammengesetzt:
1. Die Menge $L_1$ wird bestimmt, die alle Sequenzen der Länge $1$ enthält, die den minimalen Support-Wert überschreiten.
2. Solange die bereinigte Menge $L_{k-1}$ nicht leer ist,
    1. wird die Kandidatenmenge $C_k$ aus $L_{k-1}$ erzeugt und
    2. zu $L_k$ reduziert.
3. Die Vereinigung aller erzeugten Mengen häufig vorkommender Sequenzen $\bigcup_i L_i$ ist das Ergebnis.

In [ ]:
def gsp(transactions, min_supp):
    # Liste der L_k anlegen
    # mit leerer Menge als Padding
    Ls = [set(), itemsets1(transactions, min_supp)]

    # k initialisieren
    k = 2

    # Schleife
    while len(Ls[-1]) > 0:
        # Kandidatengenerierung und Filter
        C_k = generiere_kandidaten(Ls[-1], k)
        L_k = filtere_kandidaten(C_k, transactions, min_supp)

        # L_k ablegen
        Ls.append(L_k)

        # k zählen
        k += 1

    # Vereinigung bilden
    result = set()
    for L in Ls:
        result.update(L)

    return result


gsp(dna, min_supp)

## Zusammenfassung
Die Ähnlichkeit zum Apriori-Algorithmus ist offensichtlich, wobei die wichtigsten Unterschiede in der Kandidatengenerierung und dem Zählen der Vorkommen zu finden sind. Deshalb leidet der GSP-Algorithmus allerdings auch unter den selben Problemen: Es werden potentiell viele vollständige Scans der Datenbank benötigt und die Wahl des minimalen Supportwerts hat massiven Einfluss auf das Laufzeitverhalten des Algorithmus.